In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from matplotlib import rc
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import missingno as msno
from sklearn.impute import SimpleImputer
from datetime import datetime

plt.rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='NanumGothic'
#https://teddylee777.github.io/visualization/matplotlib-%EC%8B%9C%EA%B0%81%ED%99%94-%ED%95%9C%EA%B8%80%ED%8F%B0%ED%8A%B8%EC%A0%81%EC%9A%A9
%matplotlib inline 

In [28]:
import warnings
warnings.filterwarnings('ignore')

In [29]:
TA_DD_2015 = pd.read_csv("db_sfc_ta_dd2015.txt",sep=',',encoding="utf-8")
TA_DD_2016 = pd.read_csv("db_sfc_ta_dd2016.txt",sep=',',encoding="utf-8")

ta_dd_2015=TA_DD_2015.loc[:,[False,True,True,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False]]
ta_dd_2016=TA_DD_2016.loc[:,[False,True,True,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False]]

ta_dd_2015.columns =['일시','지점번호','평균기온','최고기온','최저기온']
ta_dd_2016.columns =['일시','지점번호','평균기온','최고기온','최저기온']

In [30]:
location = pd.read_csv("지점정보.csv",encoding="ansi")

df_merge5 = pd.merge(ta_dd_2015,location, left_on ='지점번호', right_on='db_comn_kma_stn.clse_tma',how='inner')
df_merge6 = pd.merge(ta_dd_2016,location, left_on ='지점번호', right_on='db_comn_kma_stn.clse_tma',how='inner')

temp_2015=df_merge5.loc[:,['일시','지점번호','평균기온','최고기온','최저기온','db_comn_kma_stn.stn_eng_nm']]
temp_2016=df_merge6.loc[:,['일시','지점번호','평균기온','최고기온','최저기온','db_comn_kma_stn.stn_eng_nm']]

temp_2015.rename(columns={'db_comn_kma_stn.stn_eng_nm':'지역'},inplace=True)
temp_2016.rename(columns={'db_comn_kma_stn.stn_eng_nm':'지역'},inplace=True)

temp_2015['일교차']=temp_2015['최고기온']-temp_2015['최저기온']
temp_2016['일교차']=temp_2016['최고기온']-temp_2016['최저기온']

temp_2015 = temp_2015[['일시','지점번호','지역','평균기온','최고기온','최저기온','일교차']]
temp_2016 = temp_2016[['일시','지점번호','지역','평균기온','최고기온','최저기온','일교차']]

temp_all = pd.concat([temp_2015,temp_2016],axis=0)

In [31]:
display(temp_all.head(5))

,일시,지점번호,지역,평균기온,최고기온,최저기온,일교차
0,2015-01-01 00:00:00.0,90,속초,-4.6,-1.8,-7.9,6.1
1,2015-01-04 00:00:00.0,90,속초,5.8,10.4,1.9,8.5
2,2015-01-09 00:00:00.0,90,속초,1.5,6.3,-2.7,9.0
3,2015-01-10 00:00:00.0,90,속초,3.1,6.4,-0.5,6.9
4,2015-01-12 00:00:00.0,90,속초,-0.2,4.9,-4.7,9.6


In [32]:
new_time =[]
for time in temp_all['일시']:
    new_time.append(time[0:10])
temp_all['일시']= new_time
temp_all.head(5)

,일시,지점번호,지역,평균기온,최고기온,최저기온,일교차
0,2015-01-01,90,속초,-4.6,-1.8,-7.9,6.1
1,2015-01-04,90,속초,5.8,10.4,1.9,8.5
2,2015-01-09,90,속초,1.5,6.3,-2.7,9.0
3,2015-01-10,90,속초,3.1,6.4,-0.5,6.9
4,2015-01-12,90,속초,-0.2,4.9,-4.7,9.6


In [33]:
temp_all["일시"] = temp_all["일시"].apply(lambda x : datetime.strptime(x,'%Y-%m-%d'))
temp_all["년"] = temp_all["일시"].apply(lambda x : x.year)
temp_all["월"] = temp_all["일시"].apply(lambda x : x.month)
temp_all["일"] = temp_all["일시"].apply(lambda x : x.day)
temp_all["일자"]=temp_all["일시"].dt.date
temp_all = temp_all[['일시','일자','년','월','일','지점번호','지역','평균기온','최고기온','최저기온','일교차']]

In [34]:
temp_all.drop(['일시'],axis=1, inplace = True)
temp_all.head(5)

,일자,년,월,일,지점번호,지역,평균기온,최고기온,최저기온,일교차
0,2015-01-01,2015,1,1,90,속초,-4.6,-1.8,-7.9,6.1
1,2015-01-04,2015,1,4,90,속초,5.8,10.4,1.9,8.5
2,2015-01-09,2015,1,9,90,속초,1.5,6.3,-2.7,9.0
3,2015-01-10,2015,1,10,90,속초,3.1,6.4,-0.5,6.9
4,2015-01-12,2015,1,12,90,속초,-0.2,4.9,-4.7,9.6


In [35]:
temp_all[temp_all["지점번호"]=="108"]

,일자,년,월,일,지점번호,지역,평균기온,최고기온,최저기온,일교차


In [36]:
loc_dic = {"강원":["강릉","대관령","동해","북강릉","속초","영월","원주","인제","정선군","철원","춘천","태백","홍천"],
            "경기":["동두천","수원","양평","이천","파주"],
            "경남":["거제", "거창", "김해시", "남해", "밀양", "북창원", "산청", "양산시", "의령군", "진주", "창원", "통영", "함양군", "합천"],
            "경북":["경주시", "구미", "문경", "봉화", "상주", "안동", "영덕", "영주", "영천", "울릉도", "울진", "의성", "청송군", "포항"],
            "광주":["광주"], "대구":["대구"],"대전":["대전"], "부산":["부산"],"서울":["관악산","서울"],"울산":["울산"],
            "인천": ["강화","백령도","인천"],
            "전남":["강진군", "고흥" ,"광양시", "목포", "보성군", "순천", "여수", "영광군", "완도", "장흥" ,"주암", "진도(첨찰산)","해남" ,"흑산도"],
            "전북":["고창", "고창군", "군산", "남원", "부안", "순창군", "임실", "장수", "전주", "정읍"],
            "제주":["고산", "서귀포", "성산","제주"],
            "충남":["금산", "보령", "부여", "서산", "천안"], "충북":["보은", "제천", "청주", "추풍령", "충주"]}

In [37]:
for key in loc_dic:
    for loc in loc_dic[key]:
        globals()["{}_df".format(loc)]=  temp_all[temp_all["지역"] == loc].sort_values(by='일자').reset_index().drop("index",axis = 1)

In [46]:
notion = ""
for key in loc_dic:
    for loc in loc_dic[key]:
        #display(globals()["{}_df".format(loc)].head(1))
        if len(globals()["{}_df".format(loc)])==0:
            notion += loc+"데이터가 없습니다.\n"
print(notion)

관악산데이터가 없습니다.
주암데이터가 없습니다.



In [17]:
강릉_df

,일자,년,월,일,지점번호,지역,평균기온,최고기온,최저기온,일교차
0,2015-01-01,2015,1,1,105,강릉,-5.2,-1.5,-8.1,6.6
1,2015-01-02,2015,1,2,105,강릉,-2.9,1.1,-6.2,7.3
2,2015-01-03,2015,1,3,105,강릉,1.4,6.6,-3.8,10.4
3,2015-01-04,2015,1,4,105,강릉,5.7,11.5,0.6,10.9
4,2015-01-05,2015,1,5,105,강릉,7.4,13.3,1.9,11.4
...,...,...,...,...,...,...,...,...,...,...
726,2016-12-27,2016,12,27,105,강릉,1.0,4.3,-2.5,6.8
727,2016-12-28,2016,12,28,105,강릉,0.5,5.8,-4.0,9.8
728,2016-12-29,2016,12,29,105,강릉,-0.4,3.9,-3.1,7.0
729,2016-12-30,2016,12,30,105,강릉,1.2,5.2,-3.0,8.2


In [18]:
for key in loc_dic:
    
    globals()["{}_df2".format(key)] = pd.DataFrame()
    region = []
    for loc in loc_dic[key]:
        globals()["{}_df2".format(key)] = pd.concat([globals()["{}_df2".format(key)],globals()["{}_df".format(loc)]],axis=0)
       
    print(key+" 데이터")
    display(globals()["{}_df2".format(key)].head(5))


강원 데이터


,일자,년,월,일,지점번호,지역,평균기온,최고기온,최저기온,일교차
0,2015-01-01,2015,1,1,105,강릉,-5.2,-1.5,-8.1,6.6
1,2015-01-02,2015,1,2,105,강릉,-2.9,1.1,-6.2,7.3
2,2015-01-03,2015,1,3,105,강릉,1.4,6.6,-3.8,10.4
3,2015-01-04,2015,1,4,105,강릉,5.7,11.5,0.6,10.9
4,2015-01-05,2015,1,5,105,강릉,7.4,13.3,1.9,11.4


경기 데이터


,일자,년,월,일,지점번호,지역,평균기온,최고기온,최저기온,일교차
0,2015-01-01,2015,1,1,98,동두천,-8.1,-2.8,-10.8,8.0
1,2015-01-02,2015,1,2,98,동두천,-7.0,-0.6,-11.8,11.2
2,2015-01-03,2015,1,3,98,동두천,-4.0,3.2,-13.0,16.2
3,2015-01-04,2015,1,4,98,동두천,1.3,7.2,-2.6,9.8
4,2015-01-05,2015,1,5,98,동두천,1.3,8.7,-5.4,14.1


경남 데이터


...

,일자,년,월,일,지점번호,지역,평균기온,최고기온,최저기온,일교차
0,2015-01-01,2015,1,1,185,고산,2.6,4.1,0.2,3.9
1,2015-01-02,2015,1,2,185,고산,5.2,8.2,3.0,5.2
2,2015-01-03,2015,1,3,185,고산,6.8,10.1,2.8,7.3
3,2015-01-04,2015,1,4,185,고산,12.1,16.3,8.4,7.9
4,2015-01-05,2015,1,5,185,고산,12.6,15.6,7.8,7.8


충남 데이터


,일자,년,월,일,지점번호,지역,평균기온,최고기온,최저기온,일교차
0,2015-01-01,2015,1,1,238,금산,-6.1,-2.7,-8.4,5.7
1,2015-01-02,2015,1,2,238,금산,-6.1,-1.2,-9.8,8.6
2,2015-01-03,2015,1,3,238,금산,-5.1,3.7,-13.2,16.9
3,2015-01-04,2015,1,4,238,금산,0.2,10.0,-5.3,15.3
4,2015-01-05,2015,1,5,238,금산,1.7,9.1,-3.1,12.2


충북 데이터


,일자,년,월,일,지점번호,지역,평균기온,최고기온,최저기온,일교차
0,2015-01-01,2015,1,1,226,보은,-6.9,-4.4,-9.2,4.8
1,2015-01-02,2015,1,2,226,보은,-6.1,-1.2,-10.3,9.1
2,2015-01-03,2015,1,3,226,보은,-6.2,2.3,-13.7,16.0
3,2015-01-04,2015,1,4,226,보은,-0.2,8.9,-5.9,14.8
4,2015-01-05,2015,1,5,226,보은,1.0,8.5,-5.3,13.8


In [39]:
def csv_to_dt(filename):
    f_read = open("{}.csv".format(filename),'r',encoding="cp949")
    lines = f_read.readlines()
    f_read.close()
    new_lines = lines[11:]

    pro_info = []#processed information
    i = 0
    for info in new_lines:
        if i ==0:
            i=1
            continue
        line = info.replace('\t','').replace('\n','').split(',')
        pro_info.append(line)
    dataframe = pd.DataFrame(pro_info)

    dataframe.columns =['지점번호','지역','일시','평균기온','최고기온','최고기온시각','최저기온','최저기온시각','일교차'] 
    dataframe.replace('None',np.nan, inplace=True)
    dataframe.replace('',np.nan, inplace=True)
    dataframe.dropna(how='all', inplace=True) # 모든 값이 결측치인 경우 제거
    dataframe = dataframe[['일시','지점번호','지역','평균기온','최고기온','최저기온','일교차']]
    return dataframe

In [40]:
new_loc = [ '파주','서울','보성군','진도(첨찰산)']

for loc in new_loc:
    globals()["{}_df".format(loc)] = csv_to_dt("{}일교차데이터2".format(loc))  

In [41]:
for loc in new_loc:
    temp_all = globals()["{}_df".format(loc)]
    temp_all["일시"] = temp_all["일시"].apply(lambda x : datetime.strptime(x,'%Y-%m-%d'))
    temp_all["년"] = temp_all["일시"].apply(lambda x : x.year)
    temp_all["월"] = temp_all["일시"].apply(lambda x : x.month)
    temp_all["일"] = temp_all["일시"].apply(lambda x : x.day)
    temp_all["일자"]=temp_all["일시"].dt.date
    temp_all = temp_all[['일자','년','월','일','지점번호','지역','평균기온','최고기온','최저기온','일교차']]
    globals()["{}_df".format(loc)] = temp_all

In [42]:
서울_df.head(3)

,일자,년,월,일,지점번호,지역,평균기온,최고기온,최저기온,일교차
0,2015-01-01,2015,1,1,108,서울,-7.7,-4.3,-9.8,5.5
1,2015-01-02,2015,1,2,108,서울,-6.0,-1.9,-9.0,7.1
2,2015-01-03,2015,1,3,108,서울,-2.7,3.1,-9.2,12.3


In [43]:
notion = ''
for key in loc_dic:    
    globals()["{}_df_all".format(key)] = pd.DataFrame()
    
    for loc in loc_dic[key]:
        globals()["{}_df_all".format(key)] = pd.concat([globals()["{}_df_all".format(key)],globals()["{}_df".format(loc)]],axis=0)
        if len(globals()["{}_df".format(loc)])==0:
            notion += loc+"데이터가 없습니다.\n"
    print(key+" 데이터")
    display(globals()["{}_df".format(key)].head(3))
print(notion)

강원 데이터


""


경기 데이터


KeyError: '경기_df'

In [44]:
for key in loc_dic:    
    globals()["{}_df".format(key)] = globals()["{}_df_all".format(key)]

KeyError: '경남_df_all'

In [45]:
강원_df.head(3)

,일자,년,월,일,지점번호,지역,평균기온,최고기온,최저기온,일교차
0,2015-01-01,2015,1,1,105,강릉,-5.2,-1.5,-8.1,6.6
1,2015-01-02,2015,1,2,105,강릉,-2.9,1.1,-6.2,7.3
2,2015-01-03,2015,1,3,105,강릉,1.4,6.6,-3.8,10.4


In [ ]:
loc_exist = ['강원','경기','경남','경북','광주','대구','대전','부산','서울','울산','인천','전남','전북','제주','충남','충북']
temp_list = ['평균기온','최고기온','최저기온','일교차']

for key in loc_exist:
    for temp in temp_list:
        globals()["{}_df".format(key)] = globals()["{}_df".format(key)].astype({temp:'float'})

In [ ]:
for key in loc_exist:
    print(key+"데이터\n")
    display(globals()["{}_df".format(key)].isnull().sum())
    fig, ax = plt.subplots(nrows=1, ncols=4, figsize = (15,5))
    i=0
    for temp in temp_list:
        df_sample1 = globals()["{}_df".format(key)][globals()["{}_df".format(key)]["년"]==2015].loc[:,temp]
        df_sample2 = globals()["{}_df".format(key)][globals()["{}_df".format(key)]["년"]==2016].loc[:,temp]
        sns.kdeplot(df_sample1, color="red", ax = ax[i] )
        sns.kdeplot(df_sample2, color="blue" ,ax = ax[i])
        i+=1
        #plt.title(temp+"데이터의 분포")
    i=0
    plt.show()

In [ ]:
for key in loc_exist:
    globals()["{}_df".format(key)].reset_index(inplace=True)

In [ ]:
for key in loc_exist:
    globals()["{}_df".format(key)].drop("index",axis =1, inplace=True)

In [ ]:
def 평균기온회귀(loc):
    loc_df = globals()["{}_df".format(loc)]
    all_no_df = loc_df.dropna(axis=0, how='any')
    평균기온 = loc_df[(loc_df["최고기온"].notnull())&(loc_df["최저기온"].notnull())&(loc_df["일교차"].notnull())]
    
    col_list = ["최고기온",'최저기온','일교차']
    X = all_no_df[col_list]
    y = all_no_df["평균기온"]
    reg = linear_model.LinearRegression()
    reg.fit(X,y)
    y_pred = reg.predict(평균기온[col_list])
    
    평균기온_pred = pd.DataFrame(평균기온[['일자','평균기온']].fillna(pd.Series(y_pred.flatten())))
    평균기온["평균기온"].fillna(pd.Series(y_pred.flatten()),inplace = True)
        
    loc_df.loc[평균기온.index, '평균기온']= loc_df.loc[평균기온.index, '평균기온'].fillna(평균기온['평균기온'])
    
    return loc_df

In [ ]:
def 최고기온회귀(loc):
    loc_df = globals()["{}_df".format(loc)]
    all_no_df = loc_df.dropna(axis=0, how='any')
    최고기온 = loc_df[(loc_df["평균기온"].notnull())&(loc_df["최저기온"].notnull())&(loc_df["일교차"].notnull())]
    
    col_list = ["평균기온",'최저기온','일교차']
    X = all_no_df[col_list]
    y = all_no_df["최고기온"]
    reg = linear_model.LinearRegression()
    reg.fit(X,y)
    y_pred = reg.predict(최고기온[col_list])
    
    최고기온_pred = pd.DataFrame(최고기온[['일자','최고기온']].fillna(pd.Series(y_pred.flatten())))
    최고기온["최고기온"].fillna(pd.Series(y_pred.flatten()),inplace = True)
    
    loc_df.loc[최고기온.index, '최고기온']= loc_df.loc[최고기온.index, '최고기온'].fillna(최고기온['최고기온'])
    
    return loc_df

In [ ]:
def 최저기온회귀(loc):
    loc_df = globals()["{}_df".format(loc)]
    all_no_df = loc_df.dropna(axis=0, how='any')
    최저기온 = loc_df[(loc_df["평균기온"].notnull())&(loc_df["최고기온"].notnull())&(loc_df["일교차"].notnull())]
    
    col_list = ["평균기온",'최고기온','일교차']
    X = all_no_df[col_list]
    y = all_no_df["최저기온"]
    reg = linear_model.LinearRegression()
    reg.fit(X,y)
    y_pred = reg.predict(최저기온[col_list])
    
    최저기온_pred = pd.DataFrame(최저기온[['일자','최저기온']].fillna(pd.Series(y_pred.flatten())))
    최저기온["최저기온"].fillna(pd.Series(y_pred.flatten()),inplace = True)
    
    loc_df.loc[최저기온.index, '최저기온']= loc_df.loc[최저기온.index, '최저기온'].fillna(최저기온['최저기온'])
    
    return loc_df

In [ ]:
loc_na_exist = ["경기","강원","충북","충남","제주","전북","전남","인천","경북","경남"]

In [ ]:
for loc in loc_na_exist:
    globals()["{}_df".format(loc)] = 평균기온회귀(loc)

In [ ]:
for loc in loc_na_exist:
    print(loc+"정보")
    display(globals()["{}_df".format(loc)].isnull().sum())

In [ ]:
loc_na_exist2 =["경기","전북","전남","인천","경북"]

In [ ]:
for loc in loc_na_exist2:
    globals()["{}_df".format(loc)] = 최고기온회귀(loc)

for loc in loc_na_exist2:
    print(loc+"정보")
    display(globals()["{}_df".format(loc)].isnull().sum())

In [ ]:
for loc in loc_na_exist2:
    globals()["{}_df".format(loc)] = 최저기온회귀(loc)
for loc in loc_na_exist2:
    print(loc+"정보")
    display(globals()["{}_df".format(loc)].isnull().sum())

In [ ]:
for key in loc_na_exist2:
    globals()["{}_df".format(key)] = globals()["{}_df".format(key)].fillna(globals()["{}_df".format(key)].rolling(window=3,min_periods =2).mean())
for key in loc_na_exist2:
    print(key+"이동평균으로 보간한 데이터\n")
    display(globals()["{}_df".format(key)].isnull().sum())

In [ ]:
for key in loc_exist:
    globals()["{}_df_final".format(key)] = globals()["{}_df".format(key)].groupby(["년","월","일"]).mean().reset_index()
    region = []

    if '지점번호' in globals()["{}_df_final".format(key)]:
       globals()["{}_df_final".format(key)].drop(['지점번호'], axis = 1, inplace=True)
    
    for i in globals()["{}_df_final".format(key)]['일']:
        region.append
        region.append(key)
    globals()["{}_df_final".format(key)]['지점명'] = region    
    display(globals()["{}_df_final".format(key)].head(5))

In [ ]:
for key in loc_exist:
    print(key+"data")
    print(globals()["{}_df_final".format(key)].isnull().sum())
    

In [ ]:
(충남_df_final+충북_df_final+대전_df_final)

In [ ]:
세종_df_final = 충남_df_final[["년","월","일","지점명"]]

In [ ]:
세종_df_final["지점명"]=세종_df_final["지점명"].replace("충남","세종")

In [ ]:
세종_df_final["평균기온"] = (충남_df_final["평균기온"]+충북_df_final["평균기온"]+대전_df_final["평균기온"])/3

In [ ]:
세종_df_final["최고기온"] = (충남_df_final["최고기온"]+충북_df_final["최고기온"]+대전_df_final["최고기온"])/3
세종_df_final["최저기온"] = (충남_df_final["최저기온"]+충북_df_final["최저기온"]+대전_df_final["최저기온"])/3
세종_df_final["일교차"] = (충남_df_final["일교차"]+충북_df_final["일교차"]+대전_df_final["일교차"])/3

In [ ]:
세종_df_final = 세종_df_final[["년","월","일","평균기온","최고기온","최저기온","일교차","지점명"]]

In [ ]:
일교차_table_all = pd.DataFrame()

In [ ]:
for loc in loc_exist:
    일교차_table_all = pd.concat([일교차_table_all,globals()["{}_df_final".format(loc)]],axis=0)

In [ ]:
일교차_table_all.to_csv('일교차_table_all.csv',encoding = 'cp949')